In [14]:
import numpy as np
from scipy.stats import multinomial
from scipy.optimize import minimize

p0 = np.array([0.09, 0.42, 0.49])
pA = np.array([0.16, 0.48, 0.36])
n = 10

# Compute the generalized likelihood ratio LR = 2(log L_hat - log L_0)
def lr_stat_full(obs):
    k = obs / np.sum(obs)  # MLE under unrestricted model
    logL_hat = np.sum(obs * np.log(k, where=(k>0)))
    logL_0   = np.sum(obs * np.log(p0, where=(obs>0)))
    return 2 * (logL_hat - logL_0)

# Monte-Carlo power simulation
def montecarlo_power(lr_stat_func, alpha=0.05, B0=200000, B1=200000):
    # 1. Simulate under H0 to find critical value
    lr_null = np.zeros(B0)
    for b in range(B0):
        obs = multinomial.rvs(n, p0)
        lr_null[b] = lr_stat_func(obs)
    c_alpha = np.quantile(lr_null, 1 - alpha)

    # 2. Simulate under alternative pA
    lr_alt = np.zeros(B1)
    for b in range(B1):
        obs = multinomial.rvs(n, pA)
        lr_alt[b] = lr_stat_func(obs)

    power = np.mean(lr_alt > c_alpha)
    return c_alpha, power

# Compute power
c1, power_full = montecarlo_power(lr_stat_full)

print("Critical value:", c1)
print("Power:", power_full)

Critical value: 5.383018599176854
Power: 0.14592


In [13]:
def hw_probs(theta):
    return np.array([theta**2, 2 * theta * (1 - theta), (1 - theta)**2])

def glrt_stat_hw(counts, theta0):
    counts = np.asarray(counts)
    if counts.ndim == 1:
        counts = counts.reshape(1, -1)   # handle single sample

    n = counts.sum(axis=1)
    x1, x2, x3 = counts[:, 0], counts[:, 1], counts[:, 2]

    # MLE of theta in HW submodel: θ̂ = (2X1 + X2)/(2n)
    theta_hat = (2 * x1 + x2) / (2 * n)

    def loglik(counts, theta):
        counts = np.asarray(counts)
        m = counts.shape[0]

        theta = np.asarray(theta)
        # If theta is scalar, broadcast to length m
        if theta.ndim == 0:
            theta = np.full(m, float(theta))

        p1 = theta**2
        p2 = 2 * theta * (1 - theta)
        p3 = (1 - theta)**2

        probs = np.stack([p1, p2, p3], axis=1)
        probs = np.clip(probs, 1e-15, 1.0)  # avoid log(0)

        return (counts * np.log(probs)).sum(axis=1)

    ll_null = loglik(counts, theta0)
    ll_mle  = loglik(counts, theta_hat)

    return 2 * (ll_mle - ll_null)

def simulate_power_hw(n=10,
                      theta0=0.3,
                      theta_a=0.4,
                      alpha=0.05,
                      n_null=100000,
                      n_alt=100000,
                      seed=0):
    rng = np.random.default_rng(seed)

    # simulate under H0 to get critical value
    p0 = hw_probs(theta0)
    counts0 = rng.multinomial(n, p0, size=n_null)
    T0 = glrt_stat_hw(counts0, theta0)
    crit = np.quantile(T0, 1 - alpha)

    # simulate under θ_a to estimate power
    pa = hw_probs(theta_a)
    countsa = rng.multinomial(n, pa, size=n_alt)
    Ta = glrt_stat_hw(countsa, theta0)
    power = np.mean(Ta > crit)

    return crit, power

if __name__ == "__main__":
    crit_value, power_hat = simulate_power_hw()
    print(f"Critical value (alpha = 0.05): {crit_value:.4f}")
    print(f"Estimated power at theta_a = 0.4: {power_hat:.4f}")


Critical value (alpha = 0.05): 4.6529
Estimated power at theta_a = 0.4: 0.1299
